In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_excel('附件2-微量成分指标数据.xlsx')

print("数据类型:")
print(df.dtypes.head(20)) 


# 将相关列转换为数值类型
for i in range(1, 18):  # F1到F17
    df.iloc[:, i] = pd.to_numeric(df.iloc[:, i], errors='coerce')
    
for i in range(32, 39):  # F33到F39
    df.iloc[:, i] = pd.to_numeric(df.iloc[:, i], errors='coerce')

# 计算总酸（第32-38列，对应F33-F39）
df['总酸_mg/L'] = df.iloc[:, 32:39].sum(axis=1)

# 计算总酯（第1-17列，对应F1-F17）
df['总酯_mg/L'] = df.iloc[:, 1:18].sum(axis=1)

# 己酸乙酯在第1列（F1）
df['己酸乙酯_mg/L'] = df.iloc[:, 1]

# 转换为g/L
df['总酸_g/L'] = df['总酸_mg/L'] / 1000
df['总酯_g/L'] = df['总酯_mg/L'] / 1000
df['己酸乙酯_g/L'] = df['己酸乙酯_mg/L'] / 1000

# 定义限值
standard_optimal = {
    '总酸': 0.4,
    '总酯': 2.0,
    '己酸乙酯': 1.2
}

standard_primary = {
    '总酸': 0.3,
    '总酯': 1.5,
    '己酸乙酯': 0.6
}

def get_grade(row):
    # 处理可能的NaN值
    if pd.isna(row['总酸_g/L']) or pd.isna(row['总酯_g/L']) or pd.isna(row['己酸乙酯_g/L']):
        return '数据缺失'
    
    if (row['总酸_g/L'] >= standard_optimal['总酸'] and
        row['总酯_g/L'] >= standard_optimal['总酯'] and
        row['己酸乙酯_g/L'] >= standard_optimal['己酸乙酯']):
        return '优级'
    elif (row['总酸_g/L'] >= standard_primary['总酸'] and
          row['总酯_g/L'] >= standard_primary['总酯'] and
          row['己酸乙酯_g/L'] >= standard_primary['己酸乙酯']):
        return '一级'
    else:
        return '不达标'

In [ ]:
df['等级'] = df.apply(get_grade, axis=1)

# 输出结果
# 找到样品编号的列，假设它在第0列
sample_id_col = df.columns[0]  # 获取第一列的列名
result = df[[sample_id_col, '总酸_g/L', '总酯_g/L', '己酸乙酯_g/L', '等级']]

# 重命名列，使结果更清晰
result.columns = ['样品编号', '总酸(g/L)', '总酯(g/L)', '己酸乙酯(g/L)', '等级']

print("结果预览:")
print(result.head())
print(f"\n总计处理了 {len(result)} 个样品")
print(f"等级分布:\n{result['等级'].value_counts()}")

# 保存结果到Excel文件
result.to_excel('白酒等级判定结果.xlsx', index=False)
print("\n结果已保存到 '白酒等级判定结果.xlsx'")